In [82]:
import pandas as pd
import feather
from pyproj import Proj, transform
import calendar as cldr
from geopy.geocoders import Nominatim
import itertools
import addfips
import zipcode
import datetime
import numpy as np

pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

### Importing feather datasets

In [83]:
# FL_df = feather.read_dataframe("feather_files\FL_raw.feather")
# VA_df = feather.read_dataframe("feather_files\VA_raw.feather")
# PA_df = feather.read_dataframe("feather_files\PA_raw.feather")
# OR_df = feather.read_dataframe("feather_files\OR_raw.feather")
# OR_loc_df = feather.read_dataframe("feather_files\OR_locations_raw.feather")
# NJ_df = feather.read_dataframe(r"feather_files\NJ_raw.feather")
# NJ_loc_df = feather.read_dataframe(r"feather_files\NJ_locations_raw.feather")
# MD_df = feather.read_dataframe("feather_files\MD_raw.feather")
# ID_df = feather.read_dataframe("feather_files\ID_raw.feather")

## PA, VA, & FL

### Scott Worland compiled the water-use data for PA, VA, and FL into the imported csv file. 

In [84]:
## PA, VA, and FL Datasets already compiled by Scott Worland
# PA_VA_FL_df = pd.read_csv("PA_VA_FL\public_supply_data_pa_va_fl.csv")

In [85]:
# PA_VA_FL_df["uid"] = PA_VA_FL_df["uid"].astype("str")

In [86]:
# PVF_df = PA_VA_FL_df.rename(columns={'mgd':'Mg'})

# FL_df = PVF_df[:].query('state == "FL"')
# VA_df = PVF_df[:].query('state == "VA"')
# PA_df = PVF_df[:].query('state == "PA"')

# feather.write_dataframe(FL_df,"feather_files\FL_raw.feather")
# feather.write_dataframe(VA_df,"feather_files\VA_raw.feather")
# feather.write_dataframe(PA_df,"feather_files\PA_raw.feather")

## FL

In [87]:
FL_df = feather.read_dataframe("feather_files\FL_raw.feather")

FL_df['month'] = FL_df["month"].astype("float")

FL_df['date'] = ""
FL_df["Mgd"]=""

FL_df['month'] = FL_df["month"].astype("int")

FL_df['date'] = FL_df.apply(lambda x: pd.datetime.strptime("{0} {1}".format(x['month'],x['year']), "%m %Y"),axis=1)

FL_df.set_index("date",inplace=True)

FL_df.rename(columns={'uid': 'id'}, inplace=True)

FL_agg=FL_df.groupby(by='id').resample('1AS').sum()

FL_agg.drop(['year','month','lat','lon'], inplace=True, axis=1)

FL_agg["Mgd"] = FL_agg["Mg"]/FL_agg["days"]

FL_agg['State']="FL"

FL_agg.reset_index(level=1, inplace=True)

FL_xy = FL_df[["id","lon","lat"]]
FL_xy.columns=['id','X_WGS','Y_WGS']
FL_xy.set_index('id',inplace=True)

FL_agg=FL_agg.join(FL_xy, how='inner')

FL_agg.reset_index(inplace=True)

FL_agg = FL_agg.groupby(['id','date']).first()

FL_agg.reset_index(inplace=True)

FL_agg.drop('days',1,inplace=True)

feather.write_dataframe(FL_agg,"feather_files\FL_agg.feather")

## VA

In [88]:
VA_df = feather.read_dataframe("feather_files\VA_raw.feather")

VA_df['date'] = ""
VA_df["Mgd"]=""
VA_df["days"]=""

VA_df['date'] = VA_df.apply(lambda x: pd.datetime.strptime("{0}".format(x['year']), "%Y"),axis=1)

VA_df['days']=VA_df.apply(lambda row: cldr.monthrange(row["year"],1)[1]
                         +cldr.monthrange(row["year"],2)[1]
                          +cldr.monthrange(row["year"],3)[1]
                          +cldr.monthrange(row["year"],4)[1]
                          +cldr.monthrange(row["year"],5)[1]
                          +cldr.monthrange(row["year"],6)[1]
                          +cldr.monthrange(row["year"],7)[1]
                          +cldr.monthrange(row["year"],8)[1]
                          +cldr.monthrange(row["year"],9)[1]
                          +cldr.monthrange(row["year"],10)[1]
                          +cldr.monthrange(row["year"],11)[1]
                          +cldr.monthrange(row["year"],12)[1],axis=1)

VA_df.set_index("date",inplace=True)

VA_df.rename(columns={'uid': 'id'}, inplace=True)

VA_agg=VA_df.groupby(by='id').resample('1AS').sum()

VA_agg.drop(['year','month','lat','lon'], inplace=True, axis=1)

VA_agg["Mgd"] = VA_agg["Mg"]/VA_agg["days"]

VA_agg['State']="VA"

VA_agg.reset_index(level=1, inplace=True)

VA_xy = VA_df[["id","lon","lat"]]
VA_xy.columns=['id','X_WGS','Y_WGS']
VA_xy.set_index('id',inplace=True)

VA_agg=VA_agg.join(VA_xy, how='inner')

VA_agg.reset_index(inplace=True)

VA_agg = VA_agg.groupby(['id','date']).first()

VA_agg.reset_index(inplace=True)

VA_agg.drop('days',1,inplace=True)

feather.write_dataframe(VA_agg,"feather_files\VA_agg.feather")

## PA

In [89]:
PA_df = feather.read_dataframe("feather_files\PA_raw.feather")

PA_df['month'] = PA_df["month"].astype("float")

PA_df['date'] = ""
PA_df["Mgd"]=""

PA_df['month'] = PA_df["month"].astype("int")

PA_df['date'] = PA_df.apply(lambda x: pd.datetime.strptime("{0} {1}".format(x['month'],x['year']), "%m %Y"),axis=1)

PA_df.set_index("date",inplace=True)

PA_df.rename(columns={'uid': 'id'}, inplace=True)

PA_agg=PA_df.groupby(by='id').resample('1AS').sum()

PA_agg.drop(['year','month','lat','lon'], inplace=True, axis=1)

PA_agg["Mgd"] = PA_agg["Mg"]/PA_agg["days"]

PA_agg['State']="PA"

PA_agg.reset_index(level=1, inplace=True)

PA_xy = PA_df[["id","lon","lat"]]
PA_xy.columns=['id','X_WGS','Y_WGS']
PA_xy.set_index('id',inplace=True)

PA_agg=PA_agg.join(PA_xy, how='inner')

PA_agg.reset_index(inplace=True)

PA_agg = PA_agg.groupby(['id','date']).first()

PA_agg.reset_index(inplace=True)

PA_agg.drop('days',1,inplace=True)

feather.write_dataframe(PA_agg,"feather_files\PA_agg.feather")

## OR

In [90]:
# # Quantity
# OR_df = pd.read_excel("OR\OWRD_PS_water_use.xlsx")

# # Location
# OR_loc_df = pd.read_excel(r"OR\OR_locations_merged.xlsx")

# feather.write_dataframe(OR_df,"feather_files\OR_raw.feather")
# feather.write_dataframe(OR_loc_df,"feather_files\OR_locations_raw.feather")

In [91]:
OR_df = feather.read_dataframe('feather_files\OR_raw.feather')
OR_loc_df = feather.read_dataframe('feather_files\OR_locations_raw.feather')

OR_df['month']=""
OR_df['year']=""
OR_df['date'] = ""
OR_df['days'] = ""
OR_df["Mgd"]=""
OR_df["Mg"]=""

def wtr_month_to_month(row):
    if row['water_month'] >3:
        return row['water_month']-3
    else:
        return row['water_month'] + 9
    
def wtr_yr_to_yr(row):
    if row['water_month'] <4:
        return row['water_year']-1
    else:
        return row['water_year']

OR_df.month = OR_df.apply(lambda row: wtr_month_to_month(row), axis=1)

OR_df.year = OR_df.apply(lambda row: wtr_yr_to_yr(row), axis=1)

OR_df['date'] = OR_df.apply(lambda x: pd.datetime.strptime("{0} {1}".format(x['month'],x['year']), "%m %Y"),axis=1)

OR_df["days"] = OR_df.apply(lambda row: cldr.monthrange(row["year"],row["month"])[1],axis=1)

OR_df.set_index("date",inplace=True)

OR_agg=OR_df.resample('1AS').sum()

OR_agg=OR_df.groupby(by='wur_report_id').resample('1AS').sum()
OR_agg["Mg"] = OR_agg["water_used (acre feet)"]*325851/1000000
OR_agg["Mgd"] = OR_agg["Mg"]/OR_agg["days"]

OR_agg.drop(['month','year','water_month','water_year','wur_report_id','water_used (acre feet)'],1,inplace=True)

OR_agg['State']="OR"

OR_agg.index.names=['id','date']

OR_loc_df = OR_loc_df.rename(columns={'wur_report_id':'id'})

OR_loc_df = OR_loc_df[['id','X_WGS','Y_WGS']]

OR_agg.reset_index(inplace=True)

OR_loc_df.reset_index(inplace=True)

OR_agg = OR_agg.merge(OR_loc_df, on='id')

OR_agg.drop(['index','days'],1,inplace=True)

feather.write_dataframe(OR_agg,"feather_files\OR_agg.feather")

## NJ

In [92]:
# # Quantity
# NJ_df = pd.read_excel("NJ\Withdrawals by HUC.xlsx")

# # Location
# NJ_loc_df = pd.read_excel(r"NJ\NJ_PS_location_data.xlsx")

# feather.write_dataframe(NJ_df,r"feather_files\NJ_raw.feather")
# feather.write_dataframe(NJ_loc_df,r"feather_files\NJ_locations_raw.feather")

In [93]:
NJ_df = feather.read_dataframe(r"feather_files\NJ_raw.feather")
NJ_loc_df = feather.read_dataframe(r"feather_files\NJ_locations_raw.feather")

NJ_df['month']=""
NJ_df['year']=""
NJ_df['date'] = ""
NJ_df['days'] = ""
NJ_df["Mgd"]=""

NJ_df['date'] = NJ_df.apply(lambda x: pd.datetime.strptime("{0} {1}".format(x['Month'],x['Year']), "%m %Y"),axis=1)

NJ_df["days"] = NJ_df.apply(lambda row: cldr.monthrange(row["Year"],row["Month"])[1],axis=1)



NJ_df.set_index('date', inplace=True)

NJ_agg=NJ_df.resample('1AS').sum()

NJ_agg=NJ_df.groupby(by='SiteName').resample('1AS').sum()

NJ_agg["Mgd"] = NJ_agg["WithdrawalMG"]/NJ_agg["days"]

NJ_agg.drop(['Month','Year','HUC14','days'],1,inplace=True)

NJ_agg['State']="NJ"

NJ_agg.index.names=['id','date']

NJ_agg.columns=[['Mg', 'Mgd', 'State']]

#NJ
outProj = Proj(proj='latlong', datum='WGS84', ellps='WGS84')
NJinProj = Proj(init='epsg:6527', preserve_units=True)
x,y = NJ_loc_df["NJEasting"].values,NJ_loc_df["NJNorthing"].values
NJ_loc_df["X_WGS"],NJ_loc_df["Y_WGS"] = transform(NJinProj,outProj,x,y)

NJ_xy = NJ_loc_df[["SiteName","X_WGS","Y_WGS"]]
NJ_xy.columns=['id','X_WGS','Y_WGS']
NJ_xy.set_index('id',inplace=True)

NJ_agg=NJ_agg.join(NJ_xy, how='inner')

NJ_agg.reset_index(inplace=True)

feather.write_dataframe(NJ_agg,r"feather_files\NJ_agg.feather")

## MD

In [94]:
# ## Quantity & location
# MD_df = pd.read_excel("MD\ScottWorland-Allsites-monthly-withdrawals-Maryland.xlsx")

# feather.write_dataframe(MD_df,r"feather_files\MD_raw.feather")

In [95]:
MD_df = feather.read_dataframe(r"feather_files\MD_raw.feather")

MD_df['Mg']=""
MD_df['Mg'] = MD_df.apply(lambda row: row['January Value']*cldr.monthrange(row["Year"],1)[1]
            +row['February Value']*cldr.monthrange(row["Year"],2)[1]
            +row['March Value']*cldr.monthrange(row["Year"],3)[1]
            +row['April Value']*cldr.monthrange(row["Year"],4)[1]
            +row['May Value']*cldr.monthrange(row["Year"],5)[1]
            +row['June Value']*cldr.monthrange(row["Year"],6)[1]
            +row['July Value']*cldr.monthrange(row["Year"],7)[1]
            +row['August Value']*cldr.monthrange(row["Year"],8)[1]
            +row['September Value']*cldr.monthrange(row["Year"],9)[1]
            +row['October Value']*cldr.monthrange(row["Year"],10)[1]
            +row['November Value']*cldr.monthrange(row["Year"],11)[1]
            +row['December Value']*cldr.monthrange(row["Year"],12)[1],axis=1)

MD_df['date'] = ""

MD_df['date'] = MD_df.apply(lambda x: pd.datetime.strptime("{0}".format(x['Year']), "%Y"),axis=1)

MD_df.set_index("date",inplace=True)

MD_df.rename(columns={'From Decimal Latitude': 'Y_WGS', 'From Decimal Longitude': 'X_WGS', 'Annual Value': 'Mgd','From Site Number': 'id'}, inplace=True)

MD_agg=MD_df.groupby(by='id').resample('1AS').sum()

MD_agg = MD_agg[['Y_WGS', 'X_WGS', 'Mgd','Mg']]

MD_agg['State']="MD"

MD_agg.reset_index(inplace=True)

feather.write_dataframe(MD_agg,r"feather_files\MD_agg.feather")

## ID

In [96]:
# ID_df = pd.read_excel("ID\Idaho-monthly-SWUDS-data.xlsx")
# ID_df["FROM_COORD_ACY_CD"] = ID_df["FROM_COORD_ACY_CD"].astype("str")
# ID_df["FROM_ALT_VA"] = ID_df["FROM_ALT_VA"].astype("str")
# ID_df["FROM_ALT_ACY_VA"] = ID_df["FROM_ALT_ACY_VA"].astype("str")

# feather.write_dataframe(ID_df,r"feather_files\ID_raw.feather")

In [97]:
ID_df = feather.read_dataframe(r"feather_files\ID_raw.feather")

ID_df['Mg']=""
ID_df['Mgd']=""
ID_df['Mg'] = ID_df.apply(lambda row: row['JAN_VAL']*cldr.monthrange(row["YEAR"],1)[1]
            +row['FEB_VAL']*cldr.monthrange(row["YEAR"],2)[1]
            +row['MAR_VAL']*cldr.monthrange(row["YEAR"],3)[1]
            +row['APR_VAL']*cldr.monthrange(row["YEAR"],4)[1]
            +row['MAY_VAL']*cldr.monthrange(row["YEAR"],5)[1]
            +row['JUN_VAL']*cldr.monthrange(row["YEAR"],6)[1]
            +row['JUL_VAL']*cldr.monthrange(row["YEAR"],7)[1]
            +row['AUG_VAL']*cldr.monthrange(row["YEAR"],8)[1]
            +row['SEP_VAL']*cldr.monthrange(row["YEAR"],9)[1]
            +row['OCT_VAL']*cldr.monthrange(row["YEAR"],10)[1]
            +row['NOV_VAL']*cldr.monthrange(row["YEAR"],11)[1]
            +row['DEC_VAL']*cldr.monthrange(row["YEAR"],12)[1], axis=1)

ID_df['date'] = ""

ID_df['Mgd']=ID_df['Mg']/ID_df.apply(lambda row: cldr.monthrange(row["YEAR"],1)[1]
            +cldr.monthrange(row["YEAR"],2)[1]
            +cldr.monthrange(row["YEAR"],3)[1]
            +cldr.monthrange(row["YEAR"],4)[1]
            +cldr.monthrange(row["YEAR"],5)[1]
            +cldr.monthrange(row["YEAR"],6)[1]
            +cldr.monthrange(row["YEAR"],7)[1]
            +cldr.monthrange(row["YEAR"],8)[1]
            +cldr.monthrange(row["YEAR"],9)[1]
            +cldr.monthrange(row["YEAR"],10)[1]
            +cldr.monthrange(row["YEAR"],11)[1]
            +cldr.monthrange(row["YEAR"],12)[1],axis=1)

ID_df['date'] = ID_df.apply(lambda x: pd.datetime.strptime("{0}".format(x['YEAR']), "%Y"),axis=1)

ID_df.set_index("date",inplace=True)

ID_df.rename(columns={'FROM_DEC_LAT_VA': 'Y_WGS', 'FROM_DEC_LONG_VA': 'X_WGS','SITE_NO': 'id'}, inplace=True)

ID_agg=ID_df[['Mgd','Mg','id']].groupby(by='id').resample('1AS').sum()

ID_agg.drop('id',1, inplace=True)

ID_agg['State']="ID"

ID_df.reset_index(inplace=True)
ID_xy = ID_df[['Y_WGS', 'X_WGS','id','date']].groupby(['id','date']).first()

ID_agg = ID_agg.join(ID_xy, how='inner')

ID_agg.reset_index(inplace=True)

feather.write_dataframe(ID_agg,r"feather_files\ID_agg.feather")

## Appending All States

In [98]:
WU_app = OR_agg
WU_app = WU_app.append([PA_agg, VA_agg, FL_agg, ID_agg, MD_agg, NJ_agg])
WU_app['id'] = WU_app['id'].astype("str")
feather.write_dataframe(WU_app,r"feather_files\WaterUse_aggregated.feather")

In [107]:
WU_app =feather.read_dataframe(r"feather_files\WaterUse_aggregated.feather")

In [106]:
WU_app

,Mg,Mgd,State,X_WGS,Y_WGS,date,id
0,32.078542,0.117504,OR,-117.220473,45.287654,2002-01-01,10591
1,40.754025,0.190439,OR,-117.220473,45.287654,2003-01-01,10591
2,49.405490,0.134988,OR,-117.220473,45.287654,2004-01-01,10591
3,180.429652,0.494328,OR,-117.220473,45.287654,2005-01-01,10591
4,360.732903,1.190538,OR,-117.220473,45.287654,2006-01-01,10591
5,65.073996,0.178285,OR,-117.220473,45.287654,2007-01-01,10591
6,67.031005,0.183145,OR,-117.220473,45.287654,2008-01-01,10591
7,48.032549,0.131596,OR,-117.220473,45.287654,2009-01-01,10591
8,126.488092,0.346543,OR,-117.220473,45.287654,2010-01-01,10591
9,51.000755,0.139728,OR,-117.220473,45.287654,2011-01-01,10591


In [100]:
from census import Census
from us import states

c = Census("fc32091d4f678e7d558e04351b3dc5f3ebb090e9")
# c.acs5.get(('NAME', 'B25034_010E'),
#           {'for': 'state:{}'.format(states.MD.fips)})

c.acs5.state(('NAME', 'B25034_010E'), states.MD.fips, year=2010)

[{'B25034_010E': '282894', 'NAME': 'Maryland', 'state': '24'}]

#### Pulling County and FIPS codes

In [101]:
geolocator = Nominatim()
# location = geolocator.reverse(list(zip(OR_df["X_WGS"],OR_df["Y_WGS"]))).raw['address']['county']
# print(location.raw['address']['county'])
x = list(zip(OR_df["X_WGS"],OR_df["Y_WGS"]))
fips = addfips.AddFIPS()
OR_df['County']=""
OR_df['zipcode']=""
OR_df['cnty_fip']=""

# for index, row in OR_df.iterrows():
#     row["County"] = lambda row: geolocator.reverse(zip(row["X_WGS"],row["Y_WGS"])).raw['address']['county']
#Then call addfips to attribute FIPS code to each county
# print(location)

KeyError: 'X_WGS'

In [ ]:
for index, row in OR_df.iterrows():
    OR_df["cnty_fip"] = fips.get_county_fips(row["county"], row['State']) 

In [ ]:
fips.get_county_fips(OR_df["county"][1], OR_df['State'][1]) 

In [ ]:
for index, row in OR_df.iterrows():
    OR_df["zipcode"] = zipcode.isinradius(zip(row["X_WGS"],row["Y_WGS"]))

### Exporting as feather files

In [ ]:
feather.write_dataframe(PA_VA_FL_df,"feather_files\PA_VA_FL.feather")
feather.write_dataframe(NJ_agg,"feather_files\NJ.feather")
feather.write_dataframe(NJ_df,"feather_files\NJ.feather")
feather.write_dataframe(NJ_loc_df,"feather_files\NJ_loc.feather")
feather.write_dataframe(OR_agg,"feather_files\OR.feather")
feather.write_dataframe(OR_df,"OR.feather")
feather.write_dataframe(OR_loc_df,"feather_files\OR_loc2.feather")
feather.write_dataframe(MD_agg2,"feather_files\MD.feather")
feather.write_dataframe(MD_df,"feather_files\MD.feather")
feather.write_dataframe(ID_df,"feather_files\ID.feather")